## August 2023

### 46. Permutations

In [ ]:
class Solution:
    def permute(self, nums: List[int]) -> List[List[int]]:
        res = []
        p = itertools.permutations(nums, len(nums))
        for pm in p:
            res.append(list(pm))
        return res

- There's not much to explain here; I used `itertools.permutations()` to get all of the permutations and then returned them.

### 17. Letter Combinations of a Phone Number

In [ ]:
class Solution:
    def letterCombinations(self, digits: str) -> List[str]:
        phonedict = dict()
        phonedict[2], phonedict[3], phonedict[4], phonedict[5], phonedict[6] = 'abc', 'def', 'ghi', 'jkl', 'mno'
        phonedict[7], phonedict[8], phonedict[9] = 'pqrs', 'tuv', 'wxyz'

        res = []
        def rec(digits, word, c, i):
            if i > len(digits):
                return
            word += c
            if len(word) == len(digits):
                res.append(word)
                return
            for c in phonedict[int(digits[i])]:
                rec(digits, word, c, i+1)
        rec(digits, '', '', 0)

        return [r for r in res if r != ""]

- First, I saved the strings corresponding to a digit on the phone in the dictionary `phonedict`.
- Then I used the recursive function `rec`, to accumulate every combination of letters for the given `digits`.  